In [ ]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the image they wish to analyse. 
# Function inputs args: None. 
# Function output 1: The file path of that which was selected by the user. 
def file_selection_dialog():
    root = Tk()
    root.title('Please select the file in question')
    root.filename = filedialog.askopenfilename(initialdir="/", title="Select A File", filetypes=[("All files", "*.*")])
    file_path = root.filename
    root.destroy()

    return file_path

In [ ]:
import pandas as pd 
from sklearn import linear_model

import ntpath
import os
import matplotlib.pyplot as plt

import numpy as np
from numpy import genfromtxt

import torch

# Function inputs args 1: file_path --> Input as string. The file path for the data in question.
# Function inputs args 2: plot_images --> Set to True or False. Wehn True, prints training data with regression line to console.
# Function inputs args 3: save_plot --> Set to True or False. When True, saves training data to file_path folder.
def MV_linear_regression(file_path, plot_images, save_plot): 

    # Import the csv file. 
    data = pd.read_csv(file_path, dtype=float)
    
    # Extract the relevant data.
    num_cols = len(data.columns)
    X = data.iloc[:, list(range(1,num_cols))]
    Y = data.iloc[:, 0]
    
    # with sklearn
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    
    ############
    # (1) Load and prepare data. 
    data = genfromtxt(file_path, delimiter=',')
    data = data[~np.isnan(data).any(axis=1)] # Remove NaN rows, such as column labels. 
    
    X = data[:, list(range(1,num_cols))]
    Y = data[:, 0]
    
    # By converting our data to float32, we reduce memory usage. 
    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)
    
    # Convert our numpy arrays to tensors. Pytorch requires the use of tensors. 
    X_tensor = torch.from_numpy(X.reshape(m,n))
    y_tensor = torch.from_numpy(y.reshape(m,1))
    
    # (1) Define our model. 
    
    